In [ ]:
######  SPARK ASSIGNMENT - P2822003 - VRETTEAS STYLIANOS ######

## PREPARATION  

# load basic packages
import findspark
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F 
# from pyspark.sql import Window


findspark.init()

spark =  SparkSession.builder.appName("test").getOrCreate()
spark

#load data
flights_data = spark.read\
                    .option("header","true")\
                    .option("inferSchema","true")\
                    .csv("671009038_T_ONTIME_REPORTING.csv")

In [ ]:
# count flights_data last check
n1 = flights_data.count()
print("number of rows all data:", n1)
n2 = flights_data.dropDuplicates().count()
print("number of rows after deleting duplicates:", n2)
n3 = n1 - n2 
print("duplicate_data:", n3)

# drop duplicates 
flights_data = flights_data.dropDuplicates()
print("number of rows final:", flights_data.count())


# display details about the dataframe 
display(flights_data)


In [ ]:
## TASK 2 


# create df2 
df2 =flights_data.select("FL_DATE","TAIL_NUM","ORIGIN","CARRIER","DEP_DELAY","DEST")
df2.show(10)


In [ ]:
# check duplicated rows 

# count Rows and Duplicate rows
n1 = df2.count()
print("number of rows df2:", n1)
n2 = df2.dropDuplicates().count()
print("number of rows after deleting duplicates:", n2)
n3 = n1 - n2 
print("duplicate_data:", n3)


In [ ]:
# drop duplicates 

df2 = df2.dropDuplicates()
#print ("number of rows:", df2.count())


In [ ]:
# check outliers

df_outliers = df2.select("ORIGIN")

df_outliers_group = df_outliers.groupBy("ORIGIN").count()

df_outliers_group.orderBy("count", ascending = True).show()


In [ ]:
# CHECK OUTLIERS in the number of flights 
outliers = df_outliers_group.approxQuantile("count", [0.01], 0.0)
outliers

In [ ]:
# rows 99%
df2.filter(~F.col("ORIGIN").isin(["AKN","PGV","GST","DLG"])).count()


In [ ]:
# rows 1%
df2.filter(F.col("ORIGIN").isin(["AKN","PGV","GST","DLG"])).count()


In [ ]:
# filter df2 ( remove the outliers)
df2 = df2.filter(~F.col("ORIGIN").isin(["AKN","PGV","GST","DLG"]))
df2.count()

In [ ]:
# GROUP BY ORIGIN - AVG_DEP_DELAY - OK
BY_ORIGIN_AVG = df2.groupBy("ORIGIN").agg(F.expr("avg(DEP_DELAY)").alias("AVG_DEP_DELAY"))

BY_ORIGIN_AVG.orderBy("AVG_DEP_DELAY", ascending = False).show()

In [ ]:
# MEDIAN FUNCTION 
median_percentile = F.expr('percentile_approx(DEP_DELAY, 0.5)')

In [ ]:
# GROUP BY ORIGIN - MEDIAN - OK
BY_ORIGIN_MED = df2.groupBy("ORIGIN").agg(median_percentile.alias("MED_DEP_DELAY"))

BY_ORIGIN_MED.orderBy("MED_DEP_DELAY", ascending = False).show()

#  median_percentile = F.expr('percentile_approx(DEP_DELAY, 0.5)')

In [ ]:
# GROUP BY CARRIER - AVG_DEP_DELAY (DRAFT) - 17 rows - OK 
BY_CARRIER_AVG = df2.groupBy("CARRIER").agg(F.expr("avg(DEP_DELAY)").alias("AVG_DEP_DELAY"))

BY_CARRIER_AVG.orderBy("AVG_DEP_DELAY", ascending = False).show()


In [ ]:
# GROUP BY CARRIER - MED_DEP_DELAY (DRAFT) - 17 rows - OK 
BY_CARRIER_MED = df2.groupBy("CARRIER").agg(median_percentile.alias("MED_DEP_DELAY"))

BY_CARRIER_MED.orderBy("MED_DEP_DELAY", ascending = False).show()

# median_percentile = F.expr('percentile_approx(DEP_DELAY, 0.5)')

In [ ]:
# BY_ORIGIN_AVG 100 first rows 

task2_ap_avg = BY_ORIGIN_AVG.limit(100)
task2_ap_avg.count()

In [ ]:
# BY_ORIGIN_MED 100 first rows

task2_ap_med = BY_ORIGIN_MED.limit(100)
task2_ap_med.count()

In [ ]:
# BY_CARRIER_AVG 100 first rows 
task2_aw_avg = BY_CARRIER_AVG.limit(100)
task2_aw_avg.count()


In [ ]:
# BY_CARRIER_MED 100 first rows
task2_aw_med = BY_CARRIER_MED.limit(100)
task2_aw_med.count()    


In [ ]:
# export csv task2_ap_avg
task2_ap_avg.limit(100).coalesce(1)\
       .write.csv("task2-ap-avg")

In [ ]:
# export csv task2_ap_med
task2_ap_med.limit(100).coalesce(1)\
       .write.csv("task2-ap-med")

In [ ]:
# export csv task2_aw_avg
task2_aw_avg.limit(100).coalesce(1)\
       .write.csv("task2-aw-avg")

In [ ]:
# export csv task2_aw_med
task2_aw_med.limit(100).coalesce(1)\
       .write.csv("task2-aw-med")